In [262]:
## gurobi
import numpy as np
from scipy import stats
import pandas as pd
from scipy.stats import t, sem

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display

import gurobipy as gp
from gurobipy import GRB

import scipy.sparse as sp
from arch import arch_model

import datetime
from datetime import timedelta

%matplotlib inline
import matplotlib.pyplot as plt

from prettytable import PrettyTable


from sympy import *

import math
from itertools import combinations,product

from IPython.display import clear_output

pd.set_option('display.max_columns', None)
pd.set_option("max_rows", 500)

print("[[IMPORT OK]]")

[[IMPORT OK]]


In [263]:
# CRE Hotel Design MINLP Optimization
# Define Scalars here
# Author: Vishal Arya
# GMU Dept Applied Math Oper Research Graduate Student

Budget = 122000000
Debt = .70
IntRate = .10 
Perdiemint = int((Budget * Debt * IntRate )/365)
bldgsqft = 100000 
Amsqft = 8000 
Restperhead = 65 
Mtgperhead = 20 
mp1 = 350 #mean
mp2 = 400
mp3 = 450
fixcostpercentrev = .30 
mq1 = 35000 #mean
mq2 = 35000
mq3 = 35000
Anncostperroom = 80000
restreq = 1 #min
mtgroomreq = 6 #max

In [264]:
#scalar dicitionaries

sqft = dict() #sq ft

sqft['single'] = 400
sqft['double'] = 500
sqft['suite'] = 600
sqft['restaurant'] = 2000
sqft['meeting'] = 500

mcost = dict() #cost per $

mcost['single'] = 500000
mcost['double'] = 625000
mcost['suite'] = 750000
mcost['restaurant'] = 1000000
mcost['meeting'] = 300000

tcost = dict() # time cost (per diem interest rate cost to build asset)

tcost['single'] = 1
tcost['double'] = 1.5
tcost['suite'] = 2
tcost['restaurant'] = 60
tcost['meeting'] = 10

In [265]:
#linear tangent approximation of price curve points defined here 

p1q, p1z, p2q, p2z, p3q, p3z, p4q, p4z, p5q, p5z, p6q, p6z = 1.5, -.5, 1.15, -.2, 1.07, 0, 1, .1, .93, .20, 0,.5


In [266]:
#Define MINLP here

m = gp.Model("hoteldesign")

#price per room type
psingle = m.addVar(name="psingle", vtype=GRB.CONTINUOUS)
pdouble = m.addVar(name="pdouble", vtype=GRB.CONTINUOUS)
psuite = m.addVar(name="psuite", vtype=GRB.CONTINUOUS)

#qty build per room type
qbsingle = m.addVar(name="qbsingle", vtype=GRB.INTEGER)
qbdouble = m.addVar(name="qbdouble", vtype=GRB.INTEGER)
qbsuite = m.addVar(name="qbsuite", vtype=GRB.INTEGER)
qbrest = m.addVar(name="qbrest", vtype=GRB.INTEGER)
qbmtg = m.addVar(name="qbmtg", vtype=GRB.INTEGER)

#qty occ per room type
qtoccsingle = m.addVar(name="qtoccsingle", vtype=GRB.INTEGER)
qtoccdouble = m.addVar(name="qtoccdouble", vtype=GRB.INTEGER)
qtoccsuite = m.addVar(name="qtoccsuite", vtype=GRB.INTEGER)

#number days to build per room type
dayssingle = m.addVar(name="dayssingle", vtype=GRB.INTEGER)
daysdouble = m.addVar(name="daysdouble", vtype=GRB.INTEGER)
dayssuite = m.addVar(name="dayssuite", vtype=GRB.INTEGER)
daysrest = m.addVar(name="daysrest", vtype=GRB.INTEGER)
daysmtg = m.addVar(name="daysmtg", vtype=GRB.INTEGER)


pcurvesingle = m.addVars(['P1z1','P1z2','P1z3','P1z4','P1z5','P1z6'], vtype=GRB.CONTINUOUS, name=['P1z1','P1z2','P1z3','P1z4','P1z5','P1z6'])
pcurvesinglebin = m.addVars(['P1y1','P1y2','P1y3','P1y4','P1y5'], vtype=GRB.BINARY, name=['P1y1','P1y2','P1y3','P1y4','P1y5'])

pcurvedouble = m.addVars(['P2z1','P2z2','P2z3','P2z4','P2z5','P2z6'], vtype=GRB.CONTINUOUS, name=['P2z1','P2z2','P2z3','P2z4','P2z5','P2z6'])
pcurvedoublebin = m.addVars(['P2y1','P2y2','P2y3','P2y4','P2y5'], vtype=GRB.BINARY, name=['P2y1','P2y2','P2y3','P2y4','P2y5'])

pcurvesuite = m.addVars(['P3z1','P3z2','P3z3','P3z4','P3z5','P3z6'], vtype=GRB.CONTINUOUS,name=['P3z1','P3z2','P3z3','P3z4','P3z5','P3z6'])
pcurvesuitebin = m.addVars(['P3y1','P3y2','P3y3','P3y4','P3y5'], vtype=GRB.BINARY, name=['P3y1','P3y2','P3y3','P3y4','P3y5'])

#set model params
m.Params.InfUnbdInfo = 1 #look for extreme rays
m.Params.NonConvex = 2 #non-convex optimization (objective function, nonlinear)

m.update()



Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1


In [267]:
#set objective function

c1 = gp.LinExpr()

c1 += mp1*(1+psingle)*qtoccsingle + mp2*(1+pdouble)*qtoccdouble + mp3*(1+psuite)*qtoccsuite + Restperhead*(qtoccsingle+qtoccdouble+qtoccsuite)*qbrest \
+ Mtgperhead*(qtoccsingle+qtoccdouble+qtoccsuite)*qbmtg #gross revenue

c1 += - Anncostperroom*(qbsingle + qbdouble + qbsuite) - fixcostpercentrev*(mp1*(1+psingle)*qtoccsingle + mp2*(1+pdouble)*qtoccdouble + mp3*(1+psuite)*qtoccsuite)


#set Objective
m.setObjective(c1, GRB.MAXIMIZE)

In [268]:
#define constraints

#sq ft constraint rooms

m.addConstr(sqft['single']*qbsingle + sqft['double']*qbdouble + sqft['suite']*qbsuite <= bldgsqft, name='sqftrooms')

#sq ft amenities constraint

m.addConstr(sqft['restaurant']*qbrest + sqft['meeting']*qbmtg <= Amsqft, name='sqftamenities')

#cost constraint

m.addConstr(mcost['single']*qbsingle + mcost['double']*qbdouble + mcost['suite']*qbsuite + mcost['restaurant']*qbrest+mcost['meeting']*qbmtg \
+ Perdiemint*(dayssingle+daysdouble + dayssuite + daysrest + daysmtg) <= Budget, name='budget')

#qty constraints

m.addConstr(qtoccsingle <= 365*qbsingle)
m.addConstr(qtoccdouble <= 365*qbdouble)
m.addConstr(qtoccsuite <= 365*qbsuite)

#linear extrapolation single price curve
m.addConstr(pcurvesingle['P1z1'] <= pcurvesinglebin['P1y1'])
m.addConstr(pcurvesingle['P1z2'] <= pcurvesinglebin['P1y1'] + pcurvesinglebin['P1y2'])
m.addConstr(pcurvesingle['P1z3'] <= pcurvesinglebin['P1y2'] + pcurvesinglebin['P1y3'])
m.addConstr(pcurvesingle['P1z4'] <= pcurvesinglebin['P1y3'] + pcurvesinglebin['P1y4'])
m.addConstr(pcurvesingle['P1z5'] <= pcurvesinglebin['P1y4'] + pcurvesinglebin['P1y5'])
m.addConstr(pcurvesingle['P1z6'] <= pcurvesinglebin['P1y5'])

#linear extrapolation dble price curve
m.addConstr(pcurvedouble['P2z1'] <= pcurvedoublebin['P2y1'])
m.addConstr(pcurvedouble['P2z2'] <= pcurvedoublebin['P2y1'] + pcurvedoublebin['P2y2'])
m.addConstr(pcurvedouble['P2z3'] <= pcurvedoublebin['P2y2'] + pcurvedoublebin['P2y3'])
m.addConstr(pcurvedouble['P2z4'] <= pcurvedoublebin['P2y3'] + pcurvedoublebin['P2y4'])
m.addConstr(pcurvedouble['P2z5'] <= pcurvedoublebin['P2y4'] + pcurvedoublebin['P2y5'])
m.addConstr(pcurvedouble['P2z6'] <= pcurvedoublebin['P2y5'])

#linear extrapolation suite price curve
m.addConstr(pcurvesuite['P3z1'] <= pcurvesuitebin['P3y1'])
m.addConstr(pcurvesuite['P3z2'] <= pcurvesuitebin['P3y1'] + pcurvesuitebin['P3y2'])
m.addConstr(pcurvesuite['P3z3'] <= pcurvesuitebin['P3y2'] + pcurvesuitebin['P3y3'])
m.addConstr(pcurvesuite['P3z4'] <= pcurvesuitebin['P3y3'] + pcurvesuitebin['P3y4'])
m.addConstr(pcurvesuite['P3z5'] <= pcurvesuitebin['P3y4'] + pcurvesuitebin['P3y5'])
m.addConstr(pcurvesuite['P3z6'] <= pcurvesuitebin['P3y5'])

clear_output()

#days build constraints
m.addConstr(tcost['single']*qbsingle == dayssingle)
m.addConstr(tcost['double']*qbdouble == daysdouble)
m.addConstr(tcost['suite']*qbsuite == dayssuite)
m.addConstr(tcost['restaurant']*qbrest == daysrest)
m.addConstr(tcost['meeting']*qbmtg == daysmtg)

#set qty curve constraint - single
m.addConstr(qtoccsingle == (p1q*pcurvesingle['P1z1'] + p2q*pcurvesingle['P1z2']+p3q*pcurvesingle['P1z3'] + \
                            p4q*pcurvesingle['P1z4']+p5q*pcurvesingle['P1z5']+p6q*pcurvesingle['P1z6'])*mq1)

m.addConstr(psingle == p1z*pcurvesingle['P1z1'] + p2z*pcurvesingle['P1z2']+p3z*pcurvesingle['P1z3'] + \
                            p4z*pcurvesingle['P1z4']+p5z*pcurvesingle['P1z5']+p6z*pcurvesingle['P1z6'])

m.addConstr(sum(pcurvesingle[i] for i in pcurvesingle.keys()) == 1)


#set qty curve constraint - double
m.addConstr(qtoccdouble == (p1q*pcurvedouble['P2z1'] + p2q*pcurvedouble['P2z2']+p3q*pcurvedouble['P2z3'] + \
                            p4q*pcurvedouble['P2z4']+p5q*pcurvedouble['P2z5']+p6q*pcurvedouble['P2z6'])*mq2)

m.addConstr(pdouble == p1z*pcurvedouble['P2z1'] + p2z*pcurvedouble['P2z2']+p3z*pcurvedouble['P2z3'] + \
                            p4z*pcurvedouble['P2z4']+p5z*pcurvedouble['P2z5']+p6z*pcurvedouble['P2z6'])

m.addConstr(sum(pcurvedouble[i] for i in pcurvedouble.keys()) == 1)


#set qty curve constraint - suite
m.addConstr(qtoccsuite == (p1q*pcurvesuite['P3z1'] + p2q*pcurvesuite['P3z2']+p3q*pcurvesuite['P3z3'] + \
                            p4q*pcurvesuite['P3z4']+p5q*pcurvesuite['P3z5']+p6q*pcurvesuite['P3z6'])*mq3)

m.addConstr(psuite == p1z*pcurvesuite['P3z1'] + p2z*pcurvesuite['P3z2']+p3z*pcurvesuite['P3z3'] + \
                            p4z*pcurvesuite['P3z4']+p5z*pcurvesuite['P3z5']+p6z*pcurvesuite['P3z6'])

m.addConstr(sum(pcurvesuite[i] for i in pcurvesuite.keys()) == 1)


#rest/mtg rm build constraints

m.addConstr(qbrest >= restreq)
m.addConstr(qbmtg <= mtgroomreq)

#
m.update()
clear_output()


In [269]:
#call optimization

m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 40 rows, 49 columns and 135 nonzeros
Model fingerprint: 0x345f78ac
Model has 9 quadratic objective terms
Variable types: 21 continuous, 28 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+06]
  Objective range  [2e+02, 8e+04]
  QObjective range [4e+01, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+08]
Presolve removed 26 rows and 20 columns
Presolve time: 0.00s
Presolved: 33 rows, 39 columns, 116 nonzeros
Presolved model has 9 bilinear constraint(s)
Variable types: 25 continuous, 14 integer (0 binary)

Root relaxation: objective 3.765570e+07, 26 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3.7656e+07    0   23          - 3.7656e+07      -  

In [270]:
m.ObjVal

26555197.5483871

In [271]:
m.getVars() #get vars


[<gurobi.Var psingle (value 0.23760368663594472)>,
 <gurobi.Var pdouble (value 0.30488479262672813)>,
 <gurobi.Var psuite (value 0.3587096774193548)>,
 <gurobi.Var qbsingle (value 78.0)>,
 <gurobi.Var qbdouble (value 58.0)>,
 <gurobi.Var qbsuite (value 42.0)>,
 <gurobi.Var qbrest (value 3.0)>,
 <gurobi.Var qbmtg (value 4.0)>,
 <gurobi.Var qtoccsingle (value 28470.0)>,
 <gurobi.Var qtoccdouble (value 21170.0)>,
 <gurobi.Var qtoccsuite (value 15330.0)>,
 <gurobi.Var dayssingle (value 78.0)>,
 <gurobi.Var daysdouble (value 87.0)>,
 <gurobi.Var dayssuite (value 84.0)>,
 <gurobi.Var daysrest (value 180.0)>,
 <gurobi.Var daysmtg (value 40.0)>,
 <gurobi.Var P1z1 (value 0.0)>,
 <gurobi.Var P1z2 (value 0.0)>,
 <gurobi.Var P1z3 (value 0.0)>,
 <gurobi.Var P1z4 (value 0.0)>,
 <gurobi.Var P1z5 (value 0.8746543778801843)>,
 <gurobi.Var P1z6 (value 0.1253456221198157)>,
 <gurobi.Var P1y1 (value 1.0)>,
 <gurobi.Var P1y2 (value 1.0)>,
 <gurobi.Var P1y3 (value 1.0)>,
 <gurobi.Var P1y4 (value 1.0)>,
 <gu

In [272]:
for i in m.getConstrs(): #print constraint name, slack, RHS
    i.ConstrName, i.Slack, i.RHS

('sqftrooms', 14600.0, 100000.0)

('sqftamenities', 0.0, 8000.0)

('budget', 76807.0, 122000000.0)

('R3', 0.0, 0.0)

('R4', 0.0, 0.0)

...

('R35', 3.637978807091713e-12, 0.0)

('R36', 5.551115123125783e-17, 0.0)

('R37', 0.0, 1.0)

('R38', -2.0, 1.0)

('R39', 2.0, 6.0)

In [273]:
#upper bound via LP optimization

r = m.relax()

In [274]:
r.update()
r.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 40 rows, 49 columns and 135 nonzeros
Model fingerprint: 0xdda316d6
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-01, 1e+06]
  Objective range  [2e+02, 8e+04]
  QObjective range [4e+01, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+08]
Presolve removed 25 rows and 20 columns

Continuous model is non-convex -- solving as a MIP.

Presolve removed 29 rows and 23 columns
Presolve time: 0.00s
Presolved: 30 rows, 36 columns, 110 nonzeros
Presolved model has 9 bilinear constraint(s)
Variable types: 36 continuous, 0 integer (0 binary)

Root relaxation: objective 3.765570e+07, 25 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3.7656e+07  

In [275]:
m.ObjVal / r.ObjVal ## note the LP relaxation is a poor indicator of the upper bound, suggest using Bender's Decomposition for problem structure 

0.9802054697092724

In [276]:
##final optimal hotel to build:

#max NOI
print("Max NOI: ${:,d} \n".format(int(round(m.ObjVal,0))))

#build reqs
print("single: ", qbsingle.X)
print("double: ", qbdouble.X)
print("suite: ", qbsuite.X)
print("Restaurants: ", qbrest.X)
print("Meeing Rooms: ", qbmtg.X)

Max NOI: $26,555,198 

single:  78.0
double:  58.0
suite:  42.0
Restaurants:  3.0
Meeing Rooms:  4.0


In [277]:
print ("[[ EXECUTION COMPLETE ]]")

[[ EXECUTION COMPLETE ]]
